# Авторазметка стилей: fastText


Альтернативный пайплайн авторазметки. Вместо zero-shot + BERT используется `fasttext.train_supervised`, 
который обучается на уверенных подсказках (`style_hint`) и переносит метки на весь корпус.

**Что делаем:**
- загружаем подготовленный датасет и фильтруем тексты по порогу уверенности hint;
- делим выборку на трейн/валидацию и обучаем fastText;
- оцениваем качество на hold-out и сравниваем с hint;
- распространяем метки на весь корпус, сохраняем псевдоразметку и кандидатов для ручной проверки.


In [1]:
# Импорты и установка fastText
import importlib
import subprocess
import sys
from pathlib import Path
from tempfile import NamedTemporaryFile

def ensure_fasttext() -> "module":
    try:
        return importlib.import_module("fasttext")
    except ModuleNotFoundError:
        candidates = [
            ("fasttext-wheel==0.9.2", []),
            ("fasttext==0.9.2", ["--no-build-isolation", "--use-pep517"])
        ]
        last_err = None
        for package, extra_args in candidates:
            try:
                subprocess.run([sys.executable, "-m", "pip", "install", package, *extra_args], check=True)
                break
            except subprocess.CalledProcessError as err:
                last_err = err
        else:
            raise ModuleNotFoundError("fasttext is not installed and installation attempts failed") from last_err
        return importlib.import_module("fasttext")

fasttext = ensure_fasttext()

import random
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

print(f"fastText version: {getattr(fasttext, '__version__', 'unknown')}")


fastText version: unknown


## Конфигурация


In [2]:
AUTOLABEL_ROOT = Path("cache_boosted/autolabel")
FASTTEXT_DIR = AUTOLABEL_ROOT / "fasttext"
FASTTEXT_DIR.mkdir(parents=True, exist_ok=True)

config = {
    "DATASET_CSV": Path("data/taiga_style_dataset.csv"),
    "TEXT_COLUMN": "text",
    "LABEL_COLUMN": "style_hint_label",
    "LABEL_NAME_MAP": {0: "разговорный стиль", 1: "официально-деловой стиль"},
    "HINT_CONFIDENCE_COLUMN": "style_hint_confidence",
    "TRAIN_CONFIDENCE_THRESHOLD": 0.7,
    "VAL_SIZE": 0.2,
    "RANDOM_SEED": 42,
    "FASTTEXT_PARAMS": {
        "epoch": 30,
        "lr": 0.5,
        "wordNgrams": 2,
        "dim": 200,
        "loss": "softmax",
    },
    "PSEUDO_MIN_CONF": 0.0,
    "UNCERTAIN_TOP_N": 300,
}

random.seed(config["RANDOM_SEED"])
np.random.seed(config["RANDOM_SEED"])


In [3]:
# Загружаем датасет и готовим признаки
data_path = config["DATASET_CSV"]
if not data_path.exists():
    raise FileNotFoundError(f"Не найден {data_path}. Сначала сформируйте корпус.")

df = pd.read_csv(data_path, encoding="utf-8")
text_col = config["TEXT_COLUMN"]
label_col = config["LABEL_COLUMN"]
hint_conf_col = config.get("HINT_CONFIDENCE_COLUMN")
label_map = config["LABEL_NAME_MAP"]

if text_col not in df.columns or label_col not in df.columns:
    raise KeyError("В датасете отсутствуют нужные столбцы")

df = df.dropna(subset=[text_col, label_col]).copy()
df[text_col] = df[text_col].astype(str).str.strip()
df = df[df[text_col].astype(bool)].reset_index(drop=True)
df[label_col] = df[label_col].astype(int)
df["hint_label_name"] = df[label_col].map(label_map)
if hint_conf_col and hint_conf_col in df.columns:
    df[hint_conf_col] = df[hint_conf_col].astype(float)

print(f"Всего записей: {len(df)}")
print(df[label_col].value_counts().rename("count"))
if hint_conf_col:
    print("Распределение уверенности hint:")
    print(df[hint_conf_col].value_counts().sort_index())


Всего записей: 16000
style_hint_label
0    8000
1    8000
Name: count, dtype: int64
Распределение уверенности hint:
style_hint_confidence
0.60    8000
0.75    8000
Name: count, dtype: int64


In [4]:
# Формируем обучающую выборку по порогу уверенности
threshold = float(config.get("TRAIN_CONFIDENCE_THRESHOLD", 0.0) or 0.0)
if hint_conf_col and threshold > 0:
    train_mask = df[hint_conf_col] >= threshold
else:
    train_mask = pd.Series(True, index=df.index)

train_df = df.loc[train_mask, [text_col, label_col]].copy()
holdout_df = df.loc[~train_mask, [text_col, label_col]].copy() if (~train_mask).any() else pd.DataFrame(columns=[text_col, label_col])

print(f"Тексты для обучения fastText: {len(train_df)} (порог={threshold})")
if not holdout_df.empty:
    print(f"Отложено без обучения: {len(holdout_df)}")

ft_train, ft_valid = train_test_split(
    train_df,
    test_size=config["VAL_SIZE"],
    random_state=config["RANDOM_SEED"],
    stratify=train_df[label_col],
)
print(f"train={len(ft_train)}, valid={len(ft_valid)}")


Тексты для обучения fastText: 8000 (порог=0.7)
Отложено без обучения: 8000
train=6400, valid=1600


In [5]:
# Утилиты для экспорта в формат fastText
def format_label(label_id: int) -> str:
    return f"__label__{label_id}"

def export_fasttext_file(frame: pd.DataFrame, prefix: str) -> Path:
    tmp = NamedTemporaryFile("w", encoding="utf-8", delete=False, suffix=f"_{prefix}.txt")
    for text, label_id in zip(frame[text_col], frame[label_col]):
        normalized = " ".join(str(text).split())
        tmp.write(f"{format_label(int(label_id))} {normalized}\n")
    tmp_path = Path(tmp.name)
    tmp.close()
    return tmp_path

train_file = export_fasttext_file(ft_train, "train")
valid_file = export_fasttext_file(ft_valid, "valid")
train_file, valid_file


(PosixPath('/tmp/tmpd7_cofuh_train.txt'),
 PosixPath('/tmp/tmpnjto8nmu_valid.txt'))

In [6]:
# Обучаем модель fastText
train_kwargs = dict(config["FASTTEXT_PARAMS"])
train_kwargs.setdefault("verbose", 2)
model = fasttext.train_supervised(input=train_file.as_posix(), **train_kwargs)
model


Read 0M words
Number of words:  57558
Number of labels: 1
Progress: 100.0% words/sec/thread:  952897 lr:  0.000000 avg.loss:  0.000000 ETA:   0h 0m 0s


In [7]:
# Оценка на валидации
def predict_ids(texts, k: int = 1):
    labels, probs = model.predict(texts, k=k)
    label_ids = [int(label[0].replace("__label__", "")) for label in labels]
    confidences = [prob[0] for prob in probs]
    return label_ids, confidences

y_true = ft_valid[label_col].to_numpy()
y_pred, y_scores = predict_ids(ft_valid[text_col].tolist())
print(f"Accuracy: {accuracy_score(y_true, y_pred):.3f}")
print(classification_report(
    y_true,
    y_pred,
    labels=sorted(label_map.keys()),
    target_names=[label_map[i] for i in sorted(label_map.keys())],
    digits=3,
))


/home/count/code/hw_nlp/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/count/code/hw_nlp/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/count/code/hw_nlp/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is"

Accuracy: 1.000
                          precision    recall  f1-score   support

       разговорный стиль      0.000     0.000     0.000         0
официально-деловой стиль      1.000     1.000     1.000      1600

                accuracy                          1.000      1600
               macro avg      0.500     0.500     0.500      1600
            weighted avg      1.000     1.000     1.000      1600



/home/count/code/hw_nlp/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/count/code/hw_nlp/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/count/code/hw_nlp/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is"

In [8]:
# Сохраняем модель и очищаем временные файлы
model_path = FASTTEXT_DIR / "fasttext_autolabel.bin"
model.save_model(model_path.as_posix())
for tmp_path in (train_file, valid_file):
    if tmp_path.exists():
        tmp_path.unlink(missing_ok=True)
model_path


PosixPath('cache_boosted/autolabel/fasttext/fasttext_autolabel.bin')

In [9]:
# Псевдорозметка всего корпуса
all_pred_ids, all_pred_scores = predict_ids(df[text_col].tolist())
pseudo_df = df.copy()
pseudo_df["pseudo_label_id"] = all_pred_ids
pseudo_df["pseudo_label_name"] = pseudo_df["pseudo_label_id"].map(label_map)
pseudo_df["pseudo_confidence"] = all_pred_scores
pseudo_df["hint_matches"] = pseudo_df[label_col] == pseudo_df["pseudo_label_id"]

mismatch_rate = 1 - pseudo_df["hint_matches"].mean()
print(f"Доля расхождений fastText vs hint: {mismatch_rate:.2%}")
pd.crosstab(pseudo_df["hint_label_name"], pseudo_df["pseudo_label_name"])


Доля расхождений fastText vs hint: 50.00%


pseudo_label_name,официально-деловой стиль
hint_label_name,
официально-деловой стиль,8000
разговорный стиль,8000


In [10]:
# Сохраняем псевдометки и список сомнительных примеров
min_conf = float(config.get("PSEUDO_MIN_CONF", 0.0) or 0.0)
pseudo_filtered = pseudo_df[pseudo_df["pseudo_confidence"] >= min_conf].copy()
if min_conf > 0:
    print(f"Отфильтровано по уверенности >= {min_conf:.2f}: {len(pseudo_filtered)} из {len(pseudo_df)}")

pseudo_path = FASTTEXT_DIR / "fasttext_pseudo_labels.csv"
pseudo_filtered.to_csv(pseudo_path, index=False)
print(f"Псевдометки сохранены в {pseudo_path}")

top_n = int(config.get("UNCERTAIN_TOP_N", 0) or 0)
if top_n > 0:
    uncertain = pseudo_filtered.nsmallest(top_n, "pseudo_confidence")
    uncertain_path = FASTTEXT_DIR / "fasttext_uncertain_samples.csv"
    uncertain.to_csv(uncertain_path, index=False)
    print(f"{len(uncertain)} примеров сохранены для ручной проверки → {uncertain_path}")


Псевдометки сохранены в cache_boosted/autolabel/fasttext/fasttext_pseudo_labels.csv
300 примеров сохранены для ручной проверки → cache_boosted/autolabel/fasttext/fasttext_uncertain_samples.csv


In [11]:
# Быстрый просмотр расхождений
pseudo_df.loc[~pseudo_df["hint_matches"], [text_col, "hint_label_name", "pseudo_label_name", "pseudo_confidence"]].head(10)


,text,hint_label_name,pseudo_label_name,pseudo_confidence
0,Кто против ЕР с сотней Казаков подъедем пошука...,разговорный стиль,официально-деловой стиль,1.00001
2,Пора прекратить размахивать Ядерной Дубиной!,разговорный стиль,официально-деловой стиль,1.00001
5,"ПРИЧИНА Россия"" - партия Путина, он ее основал...",разговорный стиль,официально-деловой стиль,1.00001
6,"На демонтаж денег нет у Интер РАО, им надо ещё...",разговорный стиль,официально-деловой стиль,1.00001
10,"""Доллар, доллар - это грязная зелёная бумажка""...",разговорный стиль,официально-деловой стиль,1.00001
15,Тупая предвыборная агитация под прикрытием суб...,разговорный стиль,официально-деловой стиль,1.00001
20,"Фаина, путину похер. А жирик отсылает нуждающи...",разговорный стиль,официально-деловой стиль,1.00001
21,"Aleksandr, что у него есть? Я к тому, что женщ...",разговорный стиль,официально-деловой стиль,1.00001
22,Дааааааа...... Это что же такое употребляет ав...,разговорный стиль,официально-деловой стиль,1.00001
27,вот как и с нами отказались встретиться руково...,разговорный стиль,официально-деловой стиль,1.00001
